In [233]:
import pandas as pd

In [234]:
SALARY_FIELD = 'Salary'

In [235]:
QUESTION_FIELD = 'MajorUndergrad'

In [236]:
SALARY_MINIMUM = 10000

In [237]:
df = pd.read_csv('./developer_survey_2017/survey_results_public.csv')

In [238]:
df.dropna(subset=[SALARY_FIELD, QUESTION_FIELD], how='any', inplace=True)

In [239]:
clickykeys = df[[SALARY_FIELD, QUESTION_FIELD]]

In [240]:
clickykeys = clickykeys[clickykeys[SALARY_FIELD] >= SALARY_MINIMUM]

In [241]:
groupby = clickykeys.groupby(QUESTION_FIELD)

In [242]:
groupby.mean().sort_values(SALARY_FIELD, ascending=False)

,Salary
MajorUndergrad,
Psychology,85644.733138
Management information systems,76538.822759
A social science,74811.018424
Fine arts or performing arts,74032.996052
A business discipline,73620.056849
Mathematics or statistics,70969.245243
A non-computer-focused engineering discipline,70839.171961
A humanities discipline,70006.308593
I never declared a major,69812.762748


In [243]:
groupby.std()

,Salary
MajorUndergrad,
A business discipline,38468.431571
A health science,27381.227730
A humanities discipline,35151.983922
A natural science,35344.509805
A non-computer-focused engineering discipline,39912.166979
A social science,33740.263409
Computer engineering or electrical/electronics engineering,39464.525631
Computer programming or Web development,27430.757009
Computer science or software engineering,37048.614924


In [244]:
groupby.median()

,Salary
MajorUndergrad,
A business discipline,68750.000000
A health science,51893.939394
A humanities discipline,61402.737048
A natural science,60948.191593
A non-computer-focused engineering discipline,64015.151515
A social science,71600.000000
Computer engineering or electrical/electronics engineering,59139.784946
Computer programming or Web development,44864.253394
Computer science or software engineering,56250.000000


In [245]:
groupby.min()

,Salary
MajorUndergrad,
A business discipline,12903.225806
A health science,13067.097343
A humanities discipline,11549.169903
A natural science,10000.000000
A non-computer-focused engineering discipline,10370.370370
A social science,11033.581961
Computer engineering or electrical/electronics engineering,10000.000000
Computer programming or Web development,10000.000000
Computer science or software engineering,10000.000000


In [246]:
groupby.max()

,Salary
MajorUndergrad,
A business discipline,185000.000000
A health science,107526.881720
A humanities discipline,180000.000000
A natural science,180000.000000
A non-computer-focused engineering discipline,190000.000000
A social science,165000.000000
Computer engineering or electrical/electronics engineering,192000.000000
Computer programming or Web development,175000.000000
Computer science or software engineering,197000.000000
